In [22]:
counter = 0

# Set UP

In [1]:
import os
from pprint import pprint
from IPython.display import Markdown, display, update_display
from langfuse.openai import AsyncOpenAI
from dotenv import load_dotenv
load_dotenv()
client = AsyncOpenAI()

# from langfuse.openai import openai
# from openai import OpenAI

In [2]:
def _msg(role, content): #returns a dict
    return {'role':role, 'content':content}

def user(content):
    return _msg('user', content)

def system(content):
    return _msg('system', content)

def assistant(content):
    return _msg('assistant', content)

# Caching

In [3]:
# import shutil
# cache.close()  # 🔴 IMPORTANT: release the file lock
# shutil.rmtree('.cache_coures_dana')

In [3]:
from diskcache import Cache
# cache = Cache() --> this is a temporary cache where it's not saved when the program is restart
cache = Cache(directory = '.cache_coures_dana') # This is not a temporary where tha cache is saved in a directory
                                                # and can be accessed any time

In [4]:
# make the caching get and set Asyc:
# wrapper thing
# when making async function: 'async' key word, when calling it, 'await' key word
import asyncio
async def set_async_cache(key, val, **kwargs): # needs key and value, and it can take other args by their key words(kwargs)
    # return sth that is awiat
    return await asyncio.to_thread(cache.set, key, val, **kwargs)

async def get_async_cache(key, **kwargs): # needs key, kwargs, and a default can be passed with kwargs
    return await asyncio.to_thread(cache.get, key, **kwargs)

# to call async func must be await

### Make the cache key with hashing

In [28]:
from hashlib import md5
import json
# print(md5(b'dana the one').hexdigest())
# md5 works with only str

def cache_key(key, **kwargs):
    kwargs_str = json.dumps(kwargs, sort_keys = True)
    kwargs_hashed = md5(kwargs_str.encode('utf-8')).hexdigest()
    hashed_key = f'{key}__{kwargs_hashed}'
    with open('log.txt', 'a') as f:
        global counter
        counter += 1
        f.write('----------\n')
        f.write('|   '+str(counter)+'   |\n')
        f.write('----------\n\n')
        f.write('logging parameters and keys:\n')
        f.write('kwargs_str:\n'+kwargs_str + '\n')
        f.write('hashed_key:\n'+hashed_key + '\n')
    return hashed_key

def _chat_completion_cache_key(*, model, messages, **kwargs): # the first * to make the function only accepts key words args
    return cache_key('openai_chat_completion', model = model, messages = messages, **kwargs)

In [6]:
from openai.types.chat import ChatCompletion
from functools import update_wrapper
CACHE_MISS_SENTINEL = object() # we need to create sth that can be created ONLY once and can't be created accidentally anytime, like a memory address => an object

async def cached_chat_completion(*, model, messages, **kwargs) -> ChatCompletion : # the 'async' because we want to use async functions
    # MAKE KEY
    cache_key = _chat_completion_cache_key(model = model, messages = messages, **kwargs)
    cached_value = await get_async_cache(cache_key, default = CACHE_MISS_SENTINEL) # we will use this as default when the key can't be found in the cache
    
    # CACHE MISS
    if cached_value is CACHE_MISS_SENTINEL:
        completion = await client.chat.completions.create(model = model, messages = messages, **kwargs) # a) make openai call
        await set_async_cache(cache_key, completion.model_dump_json()) # b) set the output in the cache
        with open('log.txt', 'a') as f:
            f.write('\nkey is not found' + '\n')
            f.write('key is set in cache' + '\n\n')
            f.write('Response:\n')
            f.write(completion.choices[0].message.content+'\n')
            f.write('------------------------------------------------------------------------------------------------------------------------------------------------\n\n')

        return completion # this is a chat completion type

    # CACHE HIT
    else:
        completion = ChatCompletion.model_validate(json.loads(cached_value))
        with open('log.txt', 'a') as f:
            f.write('\nkey is found and Response is returned' + '\n\n')
            f.write('Response:\n')
            f.write(completion.choices[0].message.content+'\n')
            f.write('------------------------------------------------------------------------------------------------------------------------------------------------\n\n')
        return completion # Cached Value is the vlue we need

cached_chat_completion = update_wrapper(cached_chat_completion, client.chat.completions.create) # for auto completion

# Project

### - Website Object

In [7]:
from bs4 import BeautifulSoup
import requests

In [9]:
# content = requests.get('https://www.linkedin.com/company/huggingface/').content
# # print('titile: ',BeautifulSoup(content, 'html.parser').title.string,'\n')
# soup = BeautifulSoup(content, 'html.parser')
# soup.title
# # soup.body(['script', 'img', 'style', 'input'])
# # for a in BeautifulSoup(content, 'html.parser').find_all('a'):
# #     print(a.get('href'))
# # [link.get('href') for link in BeautifulSoup(content, 'html.parser').find_all('a')]

In [8]:
class Website:
    url: str
    title: str
    content: str
    link: list

    def __init__(self, url):
        self.url = url

        response = requests.get(url)
        soup = BeautifulSoup(response.content, 'html.parser')
        
        self.title = soup.title.string if soup.title else 'No Title Found'

        if not soup.body(['script', 'img', 'style', 'input']):
            pass
        else:
            for irr in soup.body(['script', 'img', 'style', 'input']):
                irr.decompose()

        self.content = soup.body.get_text(strip = True, separator = '\n')

        links = [link.get('href') for link in soup.find_all('a')]
        links2 = [link for link in links if link]
        linkedin_links = [link for link in links2 if 'linkedin' in link]
        self.links = [link for link in links2 if link not in linkedin_links]

    def get_content(self):
        return f'Website Title:\n{self.title}\n\nWebsite Content:\n{self.content}\n'

In [20]:
# Website('https://huggingface.co').links

In [49]:
# webpage = Website('https://eshaaq.onl/')
# print(webpage.get_content())
# webpage.links

### - First OpenAI Call - Find Relevant Links

In [9]:
MODEL = 'gpt-4o-mini'

In [10]:
async def relative_links(url):
    system_prompt = 'You are provided with many links from a website.\
    You have to decide which links are the most relevant to that website to include in a brochure for a company.\
    such as links to an About page, or a Company page, or Careers/Jobs pages. Find all relevant links to the website\n'
    system_prompt += 'You have to respond in JSON as the output in these examples:\n'
    system_prompt += '''
    first example:{
        input: [
            'https://full.url/goes/here/about',
            'https://another.full.url/careers',
            'https://another.url/company',
            'https://another.url/category'
        ]
        output: {
            'links' : [
            {'type': 'about page', 'url': 'https://full.url/goes/here/about'},
            {'type': 'careers page', 'url': 'https://another.full.url/careers'},
            {'type': 'company page', 'url': 'https://another.url/company'},
            {'type': 'category page', 'url': 'https://another.url/category'},
            ...
            ]
        }
    }

    second example:{
        input: [
            'https://company.url/home',
            'https://company.full.url/jobs',
            'https://company.url/company-info',
            'https://company.url/twitter'
        ]
        output: {
            'links' : [
            {'type': 'home page', 'url': 'https://company.url/home'},
            {'type': 'jobs page', 'url': 'https://company.full.url/jobs'},
            {'type': 'company info page', 'url': 'https://company.url/company-info'},
            {'type': 'twitter page', 'url': 'https://company.url/twitter'},
            ...
            ]
        }
    }
    '''

    user_prompt = f'you are on the website - {url}, with title - {Website(url).title}\n'
    user_prompt += 'Decide which of these web links are relevant to that website for the brochure about the company (some might be relative links).\
    Respond with the full https URL in JSON format. Do not include Terms of Service, Privacy, email links.'
    user_prompt += f'with these links found on it:'
    user_prompt += '\n'.join(Website(url).links)
    
    response_relevant_links = await cached_chat_completion(model = MODEL,
                                                           messages = [system(system_prompt), user(user_prompt)],
                                                           response_format = {'type':'json_object'})
    return json.loads(response_relevant_links.choices[0].message.content)

In [30]:
await relative_links("https://huggingface.co")

{'links': [{'type': 'home page', 'url': 'https://huggingface.co/'},
  {'type': 'models page', 'url': 'https://huggingface.co/models'},
  {'type': 'datasets page', 'url': 'https://huggingface.co/datasets'},
  {'type': 'spaces page', 'url': 'https://huggingface.co/spaces'},
  {'type': 'documentation page', 'url': 'https://huggingface.co/docs'},
  {'type': 'enterprise page', 'url': 'https://huggingface.co/enterprise'},
  {'type': 'pricing page', 'url': 'https://huggingface.co/pricing'},
  {'type': 'join page', 'url': 'https://huggingface.co/join'},
  {'type': 'blog page', 'url': 'https://huggingface.co/blog'},
  {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'},
  {'type': 'community page', 'url': 'https://discuss.huggingface.co'},
  {'type': 'GitHub page', 'url': 'https://github.com/huggingface'},
  {'type': 'Twitter page', 'url': 'https://twitter.com/huggingface'}]}

### - Second OpenAI Call - Create The Brochure

In [31]:
async def website_details(url):
    details = Website(url).get_content()
    details += '\nRelevant links: '
    links = await relative_links(url)
    
    for link in links['links']:
        details += f"\n\n{link['type']}\n"
        details += Website(link['url']).get_content()
    return details

In [41]:
async def create_brochure(url):
    system_prompt = 'You are an assistant that analyzes the content of a webpage, its content, \
    its relative links and their contents from a company website, and creates a short brochure \
    about the company for prospective customers, investors and recruits. Respond in Markdown. \
    Include as many details as you can find.'

    user_prompt = f'You are looking at company called: {Website(url).title}, \n'
    user_prompt += 'Here are the content of its landing page and other relevant pages.\
                    Use these information to build a short brochure of the company in html,\
                    so that I can it the whole html generated text in a webpage and put all the information\
                    in a table so the reader can easily read the brochure. Decorate it so it look like it is part of the actual url webpage.\
                    Generate the html code only. Do not add any introduction or conclusion. start your response with <!DOCTYPE html> and end with "</html>"'
    user_prompt += await website_details(url)
    # user_prompt = user_prompt[:100_000] # Truncate if more than 5,000 characters

    response = await cached_chat_completion(model = MODEL,
                                 messages = [system(system_prompt), user(user_prompt)])

    # display(Markdown(response.choices[0].message.content))
    return response.choices[0].message.content

In [33]:
display(Markdown(await create_brochure("https://huggingface.co")))

```html
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Hugging Face Brochure</title>
    <style>
        body {
            font-family: Arial, sans-serif;
            margin: 0;
            padding: 20px;
            color: #333;
            background-color: #f7f7f7;
        }
        h1 {
            color: #3f51b5;
        }
        table {
            width: 100%;
            border-collapse: collapse;
            margin-bottom: 20px;
        }
        th, td {
            border: 1px solid #ddd;
            padding: 12px;
            text-align: left;
        }
        th {
            background-color: #3f51b5;
            color: white;
        }
        .header {
            text-align: center;
            margin-bottom: 30px;
        }
    </style>
</head>
<body>

    <div class="header">
        <h1>Hugging Face</h1>
        <p>The AI community building the future.</p>
    </div>

    <table>
        <tr>
            <th>Overview</th>
            <td>The platform where the machine learning community collaborates on models, datasets, and applications.</td>
        </tr>
        <tr>
            <th>Popular Services</th>
            <td>
                Models, Datasets, Spaces, Community, Documentation, Enterprise, Pricing
            </td>
        </tr>
        <tr>
            <th>Trending Models</th>
            <td>
                - deepseek-ai/DeepSeek-R1-0528 <br>
                - google/gemma-3n-E4B-it-litert-preview <br>
                - ResembleAI/chatterbox <br>
                - deepseek-ai/DeepSeek-R1-0528-Qwen3-8B <br>
                - Qwen/Qwen3-Embedding-0.6B-GGUF 
            </td>
        </tr>
        <tr>
            <th>Spaces & Applications</th>
            <td>
                - Chatterbox TTS (Expressive Zeroshot TTS) <br>
                - DeepSite (Generate applications with DeepSeek) <br>
                - Wan2.1 Fast (Turn static images into animated videos) <br>
                - Chain-of-Zoom (Super-Resolution) <br>
                - Direct3D S2 V1.0 Demo (Generate 3D models)
            </td>
        </tr>
        <tr>
            <th>Datasets</th>
            <td>
                - yandex/yambda <br>
                - open-r1/Mixture-of-Thoughts <br>
                - fka/awesome-chatgpt-prompts <br>
                - open-thoughts/OpenThoughts3-1.2M <br>
                - Hcompany/WebClick
            </td>
        </tr>
        <tr>
            <th>Enterprise Solutions</th>
            <td>
                - Starting at $20/user/month <br>
                - Enterprise-grade security <br>
                - Dedicated support <br>
                - Advanced compute options
            </td>
        </tr>
        <tr>
            <th>Open Source Projects</th>
            <td>
                - Transformers <strong>(145,283)</strong> <br>
                - Diffusers <strong>(29,263)</strong> <br>
                - Tokenizers <strong>(9,769)</strong> <br>
                - PEFT <strong>(18,689)</strong>
            </td>
        </tr>
        <tr>
            <th>Pricing</th>
            <td>
                - Free tier available <br>
                - PRO Account at $9/month <br>
                - Enterprise Hub starting at $20/user/month
            </td>
        </tr>
        <tr>
            <th>Contact & Social</th>
            <td>
                - [GitHub](https://github.com/huggingface) <br>
                - [Twitter](https://twitter.com/huggingface) <br>
                - [LinkedIn](https://linkedin.com/company/hugging-face) <br>
                - [Discord](https://discord.com/invite/huggingface)
            </td>
        </tr>
    </table>

</body>
</html>
```

In [34]:
await create_brochure("https://huggingface.co")

'```html\n<!DOCTYPE html>\n<html lang="en">\n<head>\n    <meta charset="UTF-8">\n    <meta name="viewport" content="width=device-width, initial-scale=1.0">\n    <title>Hugging Face Brochure</title>\n    <style>\n        body {\n            font-family: Arial, sans-serif;\n            background-color: #f4f4f4;\n            margin: 0;\n            padding: 20px;\n        }\n        table {\n            width: 100%;\n            border-collapse: collapse;\n            margin: 20px 0;\n            background: white;\n        }\n        th, td {\n            border: 1px solid #dddddd;\n            text-align: left;\n            padding: 8px;\n        }\n        th {\n            background-color: #007bff;\n            color: white;\n        }\n        h1 {\n            text-align: center;\n            color: #007bff;\n        }\n        h2 {\n            color: #007bff;\n        }\n    </style>\n</head>\n<body>\n\n<h1>Hugging Face - The AI Community Building the Future</h1>\n\n<table>\n   

#### - Stream Call

In [48]:
from openai import OpenAI
clientai = OpenAI()

In [49]:
async def stream_brochure(url):
    system_prompt = 'You are an assistant that analyzes the content of a webpage, its content, \
    its relative links and their contents from a company website, and creates a short brochure \
    about the company for prospective customers, investors and recruits. Respond in Markdown. \
    Include as many details as you can find.'

    user_prompt = f'You are looking at company called: {Website(url).title}, \n'
    user_prompt += 'Here are the content of its landing page and other relevant pages. Use these information to build a short brochure of the company in Markdown.'
    user_prompt += await website_details(url)
    user_prompt = user_prompt[:5_000] # Truncate if more than 5,000 characters

    stream = clientai.chat.completions.create(model = MODEL,
                                            messages = [system(system_prompt), user(user_prompt)],
                                           stream = True)
    display_handle = display(Markdown(''),display_id=True)
    response = ''
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        response = response.replace("```", "").replace('markdown', '')
        update_display(Markdown(response), display_id = display_handle.display_id)

In [19]:
await stream_brochure('https://eshaaq.onl/')

# بروشور موقع قصة عشق

---

## مقدمة

موقع قصة عشق هو منصة ترفيهية متخصصة في تقديم أحدث المسلسلات والأفلام التركية المترجمة. تسعى المنصة إلى توفير تجربة مشاهدة فريدة وممتعة لكل عشاق الدراما التركية من خلال مجموعة واسعة من المحتوى.

---

## المحتوى والعروض

### أحدث الحلقات
- **مسلسل المنظمة**: الحلقة 145
- **مسلسل دين الروح**: الحلقة 21
- **مسلسل شراب التوت**: الحلقة 102
- **مسلسل القبعة السوداء**: الحلقة 1
- **مسلسل قلب أسود**: الحلقة 33
- **مسلسل غرفة لشخصين**: الحلقة 1

### مسلسلات متاحة
- مسلسل **القبعة السوداء** 
- مسلسل **غرفة لشخصين**
- مسلسل **الغزال**
- مسلسل **حلم أشرف**
- مسلسل **السوق**
- مسلسل **كان يا مكان في اسطنبول**
- مسلسل **اذا خسر الملك**
- مسلسل **لقد اقسمت**
- مسلسل **يوم آخر**
- مسلسل **زمالة العشاق المجهولون**

### ميزات الموقع
- تقديم أحدث الحلقات بشكل مستمر.
- توفير مسلسلات تركية مترجمة كاملة.
- إمكانية البحث السريع عن الحلقات الجديدة والمفضلة.

---

## قيم الشركة ورؤيتها

تؤمن قصة عشق بأن الترفيه هو جزء أساسي من الحياة، وتعمل بجد لتوفير محتوى يجذب جميع الأذواق. تهدف الشركة إلى أن تكون الخيار الأول لعشاق الدراما التركية من خلال تقديم محتوى متنوع وجودة عالية وبتجربة مستخدم رائعة.

---

## اتصل بنا

لزيارة الموقع: [موقع قصة عشق](https://قصه-عشق.com)

جميع الحقوق محفوظة © موقع قصة عشق

---

## الختام

انضم إلى مجتمع "قصة عشق" اليوم واستمتع بأفضل ما تقدمه الدراما التركية. سواء كنت من عشاق الإثارة، الرومانسية، أو الأكشن، ستجد كل ما تريده هنا.

In [20]:
await stream_brochure('https://huggingface.co')

# Hugging Face Brochure

**Welcome to Hugging Face!**  
_Hugging Face is the AI community building the future._ Our platform is designed to facilitate collaboration among machine learning (ML) professionals, researchers, and enthusiasts. We provide an ecosystem where users can engage with models, datasets, applications, and a diverse community focused on artificial intelligence.

## 🌐 Company Overview
- **Website:** [Hugging Face](https://huggingface.co)
- **Mission:** To create a collaborative environment for the machine learning community to develop, share, and explore models and datasets effectively.

## 📊 Key Offerings
### **1. Models**
Explore over **1 million models** for a variety of applications, including:
- **ByteDance-Seed/BAGEL-7B-MoT**
- **mistralai/Devstral-Small-2505**
- **google/gemma-3n-E4B-it-litert-preview**
- **nari-labs/Dia-1.6B**

### **2. Datasets**
Access **250k+ datasets** tailored for multiple ML tasks:
- **disco-eth/EuroSpeech**
- **nvidia/OpenCodeReasoning**

### **3. Spaces**
Run applications directly within Hugging Face Spaces. Engage with over **400k applications** for various AI tasks, such as:
- **DeepSite**: Generate applications with DeepSeek
- **LTX Video Fast**: Ultra-fast video model generation
- **Joy Caption Beta One**: Image caption generation in different styles

### **4. Enterprise Solutions**
Upgrade your AI initiatives with enterprise-class tools:
- **Compute**: Optimized inference endpoints for AI applications, starting at **$0.60/hour** for GPU.
- **Enterprise Plans**: Custom solutions tailored for businesses starting at **$20/user/month**.

## 🤝 Community and Collaboration
- Join over **50,000 organizations** relying on Hugging Face, including industry leaders like:
  - **Amazon**
  - **Google**
  - **Microsoft**
  - **Intel**
  
Our community features an active feed of model and dataset updates, discussions, and collaborative opportunities.

## 📚 Open Source Framework
Our commitment to open-source is embodied in tools that serve as the foundation of ML tooling:
- **Transformers**: State-of-the-art ML for various frameworks
- **Diffusers**: Advanced diffusion models
- **Tokenizers**: Fast tokenizers optimized for research and production

## 🚀 Getting Started
Whether you're a business seeking enterprise solutions or an individual wanting to explore ML, joining Hugging Face is simple:
1. **Sign Up** for a free account to dive into our offerings.
2. **Explore** diverse ML models, datasets, and applications.
3. **Connect** with our thriving community to elevate your projects.

## 📞 Contact Us
For more information:
- Email: [contact@huggingface.co](mailto:contact@huggingface.co)
- Follow us on [GitHub](https://github.com/huggingface), [Twitter](https://twitter.com/huggingface), [LinkedIn](https://www.linkedin.com/company/huggingface/).

## Join the Future with Hugging Face!
**Build, share, and collaborate on your ML projects with us. Let’s advance AI together!**

### Third OpenAI Call - Translate the Content

In [39]:
async def create_translated_brochure(url):
    system_prompt = '''
        you are an excellent translator. You have to translate a web brochure information from english language into german proficiently.
        you will be provided by html format for a web brochure and you must generate an german version for this web page.
        You have to generate the same html format, but the information inside the page should be translated from english into german.
        You have to generate the same output as in this example:
        input:
        ```html\n<!DOCTYPE html>\n<html lang="en">\n<head>\n    <meta charset="UTF-8">\n    <meta name="viewport" content="width=device-width, initial-scale=1.0">\n    <title>Hugging Face Brochure</title>\n    <style>\n        body {\n            font-family: Arial, sans-serif;\n        }\n        h1 {\n            color: #007bff;\n        }\n    </style>\n</head>\n<body>\n\n<h1>Hugging Face - The AI Community Building the Future</h1>\n\n</body>\n</html>\n```
        output:
        ```html\n<!DOCTYPE html>\n<html lang="en">\n<head>\n    <meta charset="UTF-8">\n    <meta name="viewport" content="width=device-width, initial-scale=1.0">\n    <title>Hugging Face Broschüre</title>\n    <style>\n        body {\n            font-family: Arial, sans-serif;\n        }\n        h1 {\n            color: #007bff;\n        }\n    </style>\n</head>\n<body>\n\n<h1>Hugging Face – Die KI-Community, die die Zukunft gestaltet</h1>\n\n</body>\n</html>\n```

    '''

    user_prompt = '''
    you have the task of getting information of html formatted brochure in english language:\n
    Translate this webpage brochure from english language into german so it gives an german page.
    - Do not translate html tags, only the information between them
    - Do not translate tags like <meta>, <style>, <script> and any css or js lines\n\n
    this is the brochure:\n
    '''
    user_prompt += await create_brochure(url)
    
    response = await cached_chat_completion(model = MODEL,
                                 messages = [system(system_prompt), user(user_prompt)])

    # display(Markdown(response.choices[0].message.content))
    return response.choices[0].message.content

In [46]:
display(Markdown(await create_translated_brochure("https://huggingface.co")))

```html
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Hugging Face - Unternehmensbroschüre</title>
    <style>
        body {
            font-family: Arial, sans-serif;
            background-color: #f5f5f5;
            color: #333;
            padding: 20px;
        }
        table {
            width: 100%;
            border-collapse: collapse;
            margin-bottom: 20px;
        }
        th, td {
            border: 1px solid #ddd;
            padding: 10px;
            text-align: left;
        }
        th {
            background-color: #4CAF50;
            color: white;
        }
    </style>
</head>
<body>

<h1>Hugging Face - Die KI-Community, die die Zukunft gestaltet</h1>

<table>
    <tr>
        <th>Website-Inhalt</th>
        <th>Beschreibung</th>
    </tr>
    <tr>
        <td>Modelle</td>
        <td>Zugriff auf über 1 Million Modelle, mit angesagten Modellen wie <strong>deepseek-ai/DeepSeek-R1-0528</strong> und <strong>google/gemma-3n-E4B-it-litert-preview</strong>.</td>
    </tr>
    <tr>
        <td>Datensätze</td>
        <td>Durchsuchen Sie über 250.000 Datensätze, darunter <strong>yandex/yambda</strong> und <strong>open-r1/Mixture-of-Thoughts</strong>.</td>
    </tr>
    <tr>
        <td>Spaces</td>
        <td>Entdecken Sie Anwendungen für verschiedene Aufgaben wie Textgenerierung und Bildbearbeitung mit über 400.000 verfügbaren Anwendungen.</td>
    </tr>
    <tr>
        <td>Community</td>
        <td>Schließen Sie sich über 50.000 Organisationen an, die Hugging Face für KI-Lösungen nutzen, darunter Giganten wie Google, Amazon und Microsoft.</td>
    </tr>
    <tr>
        <td>Unternehmenslösungen</td>
        <td>Skalierbare Lösungen ab 20 $/Benutzer/Monat mit Unternehmenssicherheit und dediziertem Support.</td>
    </tr>
    <tr>
        <td>Open-Source-Tools</td>
        <td>Innovative Tools wie <strong>Transformers</strong>, <strong>Diffusers</strong> und <strong>Safetensors</strong> für den Bau von Machine-Learning-Anwendungen.</td>
    </tr>
    <tr>
        <td>Preise</td>
        <td>Kostenloses Basismodell verfügbar, mit PRO-Konten für 9 $/Monat und Unternehmenslösungen ab 20 $/Benutzer/Monat für Unternehmen.</td>
    </tr>
</table>

<h2>Angesagte Modelle Diese Woche</h2>
<table>
    <tr>
        <th>Modellname</th>
        <th>Letzte Aktualisierung</th>
        <th>Likes</th>
        <th>Kommentare</th>
    </tr>
    <tr>
        <td>deepseek-ai/DeepSeek-R1-0528</td>
        <td>Vor 9 Tagen</td>
        <td>82.1k</td>
        <td>1.83k</td>
    </tr>
    <tr>
        <td>google/gemma-3n-E4B-it-litert-preview</td>
        <td>Vor 12 Tagen</td>
        <td>993</td>
        <td></td>
    </tr>
    <tr>
        <td>ResembleAI/chatterbox</td>
        <td>Vor 9 Tagen</td>
        <td>678</td>
        <td></td>
    </tr>
    <tr>
        <td>Qwen/Qwen3-Embedding-0.6B-GGUF</td>
        <td>Vor 1 Tag</td>
        <td>5.04k</td>
        <td>210</td>
    </tr>
</table>

<h2>Unternehmenskunden</h2>
<ul>
    <li>Amazon</li>
    <li>Google</li>
    <li>Microsoft</li>
    <li>Grammarly</li>
    <li>Meta</li>
</ul>

</body>
</html>
```

#### - Steeam Call

In [52]:
async def stream_german_brochure(url):
    system_prompt = '''
        you are an excellent translator. You have to translate a web brochure information from english language into german proficiently.
        you will be provided by html format for a web brochure and you must generate an german version for this web page.
        You have to generate the same html format, but the information inside the page should be translated from english into german.
        You have to generate the same output as in this example:
        input:
        ```html\n<!DOCTYPE html>\n<html lang="en">\n<head>\n    <meta charset="UTF-8">\n    <meta name="viewport" content="width=device-width, initial-scale=1.0">\n    <title>Hugging Face Brochure</title>\n    <style>\n        body {\n            font-family: Arial, sans-serif;\n        }\n        h1 {\n            color: #007bff;\n        }\n    </style>\n</head>\n<body>\n\n<h1>Hugging Face - The AI Community Building the Future</h1>\n\n</body>\n</html>\n```
        output:
        ```html\n<!DOCTYPE html>\n<html lang="en">\n<head>\n    <meta charset="UTF-8">\n    <meta name="viewport" content="width=device-width, initial-scale=1.0">\n    <title>Hugging Face Broschüre</title>\n    <style>\n        body {\n            font-family: Arial, sans-serif;\n        }\n        h1 {\n            color: #007bff;\n        }\n    </style>\n</head>\n<body>\n\n<h1>Hugging Face – Die KI-Community, die die Zukunft gestaltet</h1>\n\n</body>\n</html>\n```

    '''

    user_prompt = '''
    you have the task of getting information of html formatted brochure in english language:\n
    Translate this webpage brochure from english language into german so it gives an german page.
    - Do not translate html tags, only the information between them
    - Do not translate tags like <meta>, <style>, <script> and any css or js lines but include them in the html code\n\n
    this is the brochure:\n
    '''
    user_prompt += await create_translated_brochure(url)

    stream = clientai.chat.completions.create(model = MODEL,
                                            messages = [system(system_prompt), user(user_prompt)],
                                           stream = True)
    display_handle = display(Markdown(''),display_id=True)
    response = ''
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        response = response.replace("```", "").replace('markdown', '')
        update_display(Markdown(response), display_id = display_handle.display_id)

In [53]:
await stream_german_brochure('https://huggingface.co')

html
<!DOCTYPE html>
<html lang="de">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Hugging Face - Broschüre</title>
    <style>
        body {
            font-family: Arial, sans-serif;
            background-color: #f4f4f4;
            color: #333;
            margin: 0;
            padding: 0;
        }
        .container {
            width: 80%;
            margin: auto;
            overflow: hidden;
            background: white;
            padding: 20px;
            border-radius: 8px;
            box-shadow: 0 0 10px rgba(0, 0, 0, 0.1);
        }
        h1, h2, h3 {
            color: #1a1a1a;
        }
        table {
            width: 100%;
            border-collapse: collapse;
            margin: 20px 0;
        }
        th, td {
            padding: 12px;
            border: 1px solid #ddd;
            text-align: left;
        }
        th {
            background-color: #007bff;
            color: white;
        }
        footer {
            text-align: center;
            margin-top: 20px;
        }
    </style>
</head>
<body>
    <div class="container">
        <h1>Hugging Face</h1>
        <p>Die KI-Community, die die Zukunft gestaltet. Die Plattform, auf der die Machine-Learning-Community an Modellen, Datensätzen und Anwendungen zusammenarbeitet.</p>

        <h2>Wichtige Angebote</h2>
        <table>
            <tr>
                <th>Funktion</th>
                <th>Beschreibung</th>
            </tr>
            <tr>
                <td>Modelle</td>
                <td>Zusammenarbeit an unbegrenzt öffentlichen Modellen. Zugriff auf über 1 Million Modelle.</td>
            </tr>
            <tr>
                <td>Datensätze</td>
                <td>Zugriff auf und Teilen von Datensätzen für beliebige ML-Aufgaben. Über 250.000 Datensätze verfügbar.</td>
            </tr>
            <tr>
                <td>Spaces</td>
                <td>Führen Sie KI-Anwendungen wie Bild- und Textgenerierung einfach aus.</td>
            </tr>
            <tr>
                <td>Enterprise</td>
                <td>Erweiterte Plattform für den Bau von KI mit Sicherheit und Zugriffskontrollen.</td>
            </tr>
            <tr>
                <td>Preise</td>
                <td>Ab der kostenlosen Zusammenarbeit, mit PRO-Konten für 9 $/Monat und Enterprise ab 20 $/Benutzer/Monat.</td>
            </tr>
        </table>

        <h2>Beliebte Modelle</h2>
        <table>
            <tr>
                <th>Modellname</th>
                <th>Aktualisierungen</th>
                <th>Likes</th>
                <th>Follower</th>
            </tr>
            <tr>
                <td>deepseek-ai/DeepSeek-R1-0528</td>
                <td>Vor 9 Tagen aktualisiert</td>
                <td>82.1k</td>
                <td>1.83k</td>
            </tr>
            <tr>
                <td>google/gemma-3n-E4B-it-litert-preview</td>
                <td>Vor 12 Tagen aktualisiert</td>
                <td>993</td>
                <td>---</td>
            </tr>
            <tr>
                <td>ResembleAI/chatterbox</td>
                <td>Vor 9 Tagen aktualisiert</td>
                <td>678</td>
                <td>---</td>
            </tr>
        </table>

        <h2>Enterprise-Funktionen</h2>
        <table>
            <tr>
                <th>Funktion</th>
                <th>Beschreibung</th>
            </tr>
            <tr>
                <td>Single Sign-On</td>
                <td>Stellen Sie sicher, dass Sie sich mit Ihrem Identitätsanbieter verbinden.</td>
            </tr>
            <tr>
                <td>Audit-Logs</td>
                <td>Umfassende Protokolle, die über getätigte Aktionen berichten.</td>
            </tr>
            <tr>
                <td>Ressourcengruppen</td>
                <td>Granulare Zugriffskontrolle für das Management von Repositories.</td>
            </tr>
            <tr>
                <td>Prioritätsunterstützung</td>
                <td>Maximieren Sie Ihre Plattformnutzung mit priorisierter Unterstützung von Hugging Face.</td>
            </tr>
        </table>

        <footer>
            <p>Verbinden Sie sich mit uns auf <a href="https://twitter.com/huggingface" target="_blank">Twitter</a></p>
            <p>Besuchen Sie unsere <a href="https://huggingface.co/" target="_blank">Website</a> für weitere Details.</p>
        </footer>
    </div>
</body>
</html>
